In [1]:
# dependencies
import json
import pandas as pd
import re

# load types file
with open('../API_Data/pkmn.json', 'r') as in_file:
    pkmn = json.load(in_file)

In [81]:
pkmn_names = pd.Series([pkmn[num]['name'] for num in range(len(pkmn))])

Here I'm going to want to change out form names to something that looks a little better (ex: raichu-alola -> Alolan Raichu)

In [82]:
def clean_names(name):

    # capture all region-specific forms
    regions = ['alol', 'galar', 'hisui']
    for region in regions:
        if re.match(f'.*{region}', name, flags=re.IGNORECASE):
            
            new = re.search(f'(.*)-{region}', name)
            if new:
                if region in ['alol', 'hisui']:
                    name = f'{region}an {new.group(1)}'
                elif region in ['galar']:
                    name = f'{region}ian {new.group(1)}'
            return name
    
    # list of things to drop
    # gigantimaxes, pikachu and eevee costumes
    to_drop = ['gmax', 'pikachu-', 'eevee-']
    for pattern in to_drop:
        if re.match(f'.*{pattern}', name, flags = re.IGNORECASE):
            name = 'DROP'
            return name

    # pokemon with different formes
    forme_search = ['incarnate', 'therian', 'normal', 'attack', 'defense', 'speed', 'origin', 'altered', 'land', 'aria', 'pirouette', 'shield', 'blade']
    for pattern in forme_search:
        if re.match(f'.*-{pattern}$', name, flags = re.IGNORECASE):
            new = re.search(f'(.*)-{pattern}', name)
            name = f'{new.group(1)} {pattern} forme'
            return name
    
    # pokemon with different forms
    form_search = ['sunny', 'rainy', 'snowy', 'ordinary', 'resolute', 'midday', 'midnight', 'dusk', 'solo', 'school', 'amped', 'low-key']
    for pattern in form_search:
        if re.match(f'.*-{pattern}$', name, flags = re.IGNORECASE):
            new = re.search(f'(.*)-{pattern}', name)
            name = f'{new.group(1)} {pattern} form'
            return name

    # capture all mega evolutions
    if re.match('.*mega', name, flags=re.IGNORECASE):
        
        if re.match('.*-mega$', name, flags=re.IGNORECASE):
            new = re.search(r'(.*)-mega', name)
            if new:
                name = 'mega ' + new.group(1)
        elif re.match('.*-mega-[xy]', name, flags=re.IGNORECASE):
            new = re.search(r'(.*)-mega-([xy])', name)
            if new:
                name = 'mega ' + new.group(1) + ' ' + new.group(2)
            return name
        
    return name


pkmn_names = pkmn_names.apply(clean_names)

In [89]:
for name in pkmn_names[pkmn_names.str.contains('dusk')]:
    print(name)

duskull
dusknoir
lycanroc dusk form
necrozma dusk form


Pokemon names to leave alone:
Ho-Oh
Jangmo-o
Hakamo-o
Kommo-o

Single edits:
type-null -> Type: Null
Zygarde (forme)
Rotom
Basculin (form) (add striped forms and replace - with a space?)
Minior (form)
Toxtricity Low-Key (form)
Necrozma (do this before the "dusk" search)